## 1. ОБРАБОТКА ДАННЫХ ИЗ РЕЕСТРА

### 1.1 Настройка библиотек

#### 1.1.1 Библиотеки для работы с директориями

In [ ]:
import os
import sys
import csv

#### 1.1.2 Библиотеки для работы с данными

In [ ]:
# Для обработки таблиц и работы с массивами
import re
import numpy as np
import pandas as pd
import openpyxl
import json

import warnings

# Отключение предупреждений, возникающих при чтении данных
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl.styles.stylesheet')

#### 1.1.3 Прочие библиотеки

In [ ]:
from datetime import datetime

### 1.2 Настройка директорий

In [ ]:
this_directory = os.getcwd()

dipl_directory = this_directory.split('main_folder')[0]
data_directory = dipl_directory + 'raw_data'

# В WIN подключение к данным выглядит так:
xlsx_directory = data_directory + '\\excel\\'
dcsv_directory = data_directory + '\\csv\\'

# Директория выгруженных и первично обработанных данных
data_clean_directory = this_directory.split('notebooks')[0] + 'data'

### 1.3 Пересохранение данных из excel в csv (если это требуется)

In [ ]:
# Функция пересохранения файла
def excel_to_csv(name_file_excel: str, page_save: str, name_file_csv: str):
    file_open = pd.read_excel(name_file_excel, sheet_name=page_save)
    file_open.to_csv(name_file_csv, index=False)

In [ ]:
# M_name = 'reestr_re_sold_M.xlsx'
# MO_name = 'reestr_re_sold_MO.xlsx'
# NM_name = 'reestr_re_sold_NM.xlsx'
# SPB_name = 'reestr_re_sold_SPB.xlsx'
# LO_name = 'reestr_re_sold_LO.xlsx'

# # Структура данных из реестра:
# # 1. (reestr_re_sold_M) Москва - объём проданных площадей, контрактация и описательная статистика ЖК 
# # 2. (reestr_re_sold_MO) Московская область - объём проданных площадей, контрактация и описательная статистика ЖК
# # 3. (reestr_re_sold_NM) Новая Москва - объём проданных площадей, контрактация и описательная статистика ЖК
# # 4. (reestr_re_sold_SPB) Санкт-Петербург - объём проданных площадей, контрактация и описательная статистика ЖК
# # 5. (reestr_re_sold_LO) Ленинградская область - объём проданных площадей, контрактация и описательная статистика ЖК
# excel_to_csv(xlsx_directory + M_name, 'Данные', dcsv_directory + 'realty_sold_M.csv')
# excel_to_csv(xlsx_directory + MO_name, 'Данные', dcsv_directory + 'realty_sold_MO.csv')
# excel_to_csv(xlsx_directory + NM_name, 'Данные', dcsv_directory + 'realty_sold_NM.csv')
# excel_to_csv(xlsx_directory + SPB_name, 'Данные', dcsv_directory + 'realty_sold_SPB.csv')
# excel_to_csv(xlsx_directory + LO_name, 'Данные', dcsv_directory + 'realty_sold_LO.csv')

### 1.4 Первичная обработка данных

In [ ]:
os.chdir(dcsv_directory)

csv_msk = pd.read_csv('realty_sold_M.csv', low_memory=False)
csv_mo = pd.read_csv('realty_sold_MO.csv', low_memory=False)
csv_nm = pd.read_csv('realty_sold_NM.csv', low_memory=False)
csv_spb = pd.read_csv('realty_sold_SPB.csv', low_memory=False)
csv_lo = pd.read_csv('realty_sold_LO.csv', low_memory=False)

In [ ]:
lst_tbls = [csv_msk.columns, csv_mo.columns, csv_nm.columns, csv_spb.columns, csv_lo.columns]

#### 1.4.1 Установка временного промежутка  

Простая функция обработки дат и отбор от 2018 года

In [ ]:
def first_prepare(df: object) -> object:
    
    # Обработка дат
    start_date = datetime(2018, 1, 1)
    end_date = datetime(2025, 1, 1)

    df['Дата регистрации'] = pd.to_datetime(df['Дата регистрации'])
    new_df = df[(df['Дата регистрации'] >= start_date) & (df['Дата регистрации'] < end_date)]
    
    return new_df

Список параметров, которые оставляем в таблице для обработки и использования

In [ ]:
columns_select = [
                'ID ЖК',                            # обработка не требуется
                'ЖК рус',                           # обработано
                'Цена со скидкой',                  # обработка не требуется
                'Район Город',                      # обработано
                'Регион',                           # обработано (на случай конката таблиц из разных регионов)
                'Застройщик ЖК',                    # обработано
                'Площадь',                          # обработка не требуется
                'Тип Комнатности',                  # обработано
                'Этаж',                             # обработка не требуется
                'Тип помещения',                    # обработано
                'Дата регистрации',                 # обработано
                'Условия обременения',              # обработано
                'Дата обременения',                 # обработано
                'Залогодержатель',                  # обработано             
                'Длительность обременения',         # обработано
                'Тип обременения',                  # обработано
                'Дата ДДУ',                         # обработано
                'Уступка',                          # обработка не требуется
                'Купил лотов в ЖК',                 # обработано
                'Покупатель ЮЛ',                    # ВАЖНО
                'класс',                            # обработано
                'Срок сдачи',                       
                'lat',                              # обработка не требуется
                'lng',                              # обработка не требуется
                'Ипотека',                          # обработка не требуется
                'Отделка',                          # обработано
                'Старт продаж К',                   # обработано
                # 'Продавец ЮЛ',
                'Зона',                             # обработано
                # 'Купил кв и ап в ЖК',             
                'Стадия строительства в дату ДДУ'   # обработано
                ]

In [ ]:
# --------------------------------------------------------
# Обработка таблиц на даты
msk_date_update = first_prepare(csv_msk)
mo_date_update = first_prepare(csv_mo)
nm_date_update = first_prepare(csv_nm)
lo_date_update = first_prepare(csv_lo)
spb_date_update = first_prepare(csv_spb)

# Очистка памяти
csv_msk = None
csv_mo = None
csv_nm = None
csv_lo = None
csv_spb = None

# --------------------------------------------------------
# Обработка таблиц на параметры
new_msk = msk_date_update[columns_select]
new_mo = mo_date_update[columns_select]
new_nm = nm_date_update[columns_select]
new_lo = lo_date_update[columns_select]
new_spb = spb_date_update[columns_select]

# Повторная очистка памяти
msk_date_update = None
mo_date_update = None
nm_date_update = None
lo_date_update = None
spb_date_update = None

#### 1.4.2 Настройка длительности обременения

In [ ]:
'''
ФУНКЦИЯ ИЗВЛЕЧЕНИЯ ДЛИТЕЛЬНОСТИ ОБРЕМЕНЕНИЯ ИЗ ТЕКСТА
Возвращает:
- Количество месяцев как целое число
- None если не смогла распознать параметр
'''
def extract_duration_months(text, mortgage_date):

    text = str(text).lower().strip()
    
    # Кейс 1: чёткое задано количество месяцев
    month_patterns = [
                    r'(\d+)\s*(?:мес|месяц|месяца|месяцев|мес\.|\bмес\b)',
                    r'на\s*(\d+)\s*месяц',
                    r'(\d+)\s*калер?нд?а?р?н?ы?х?\s*(?:месяц|дн)',
                    r'(\d+)\s*процент(?:ных|ый|ых)\s*период',
                    r'(\d+)\s*%\s*период',
                    r'(\d+)\s*vtczw',
                    r'(\d+)\s*(?:иес|есяц|сес|инс|дн|дней)',
                    r'срок\s*(?:кредита|займа|возврата)\s*(\d+)',
                    r'срок\s*(?:кредита|займа|возврата)[^0-9]+(\d+)',
                    r'на\s*срок\s*(\d+)',
                    r'на\s*(\d+)',
                    r'последнее\s*число\s*(\d+)',
                    r'по\s*(?:последнее\s*число|последнее)\s*(\d+)',
                    r'по\s*(\d+)[\s\-]*(?:е|ое|го)',
                    r'(\d+)[\s\-]*(?:ого|его|го|ый|ий|й)',
                    ]

    for pattern in month_patterns:
        match = re.search(pattern, text)
        if match:
            try:
                num = int(match.group(1))
                # Если число слишком большое (например, 2340), вероятно это опечатка
                if num > 1000 and num < 10000:
                    # Попробуем понять, что имелось в виду
                    if num // 100 <= 36:    # максимум 36 лет (432 месяца)
                        return num // 10    # например, 2340 -> 234 месяцев
                    else:
                        return num // 100   # например, 6385 -> 63 месяца
                return num
            except:
                pass

    # Проверка на конкретные числа месяцев в тексте
    if re.search(r'\b360\b', text) or "триста шестьдесят" in text:
        return 360
    elif re.search(r'\b300\b', text) or "триста" in text:
        return 300
    elif re.search(r'\b240\b', text) or "двести сорок" in text:
        return 240
    elif re.search(r'\b180\b', text) or "сто восемьдесят" in text:
        return 180
    elif re.search(r'\b144\b', text) or "сто сорок четыре" in text:
        return 144
    elif re.search(r'\b120\b', text) or "сто двадцать" in text:
        return 120
    elif re.search(r'\b96\b', text) or "девяносто шесть" in text:
        return 96
    elif re.search(r'\b84\b', text) or "восемьдесят четыре" in text:
        return 84
    
    # Кейс 2: дата до конкретной даты
    date_patterns = [
                    r'до\s*(\d{1,2})\./-](\d{1,2})[\./-',
                    r'до\s*(\d{1,2})\s*(\w+)\s*(\d{4})',
                    r'по\s*(\d{1,2})\./-](\d{1,2})[\./-',
                    r'по\s*(\d{1,2})\s*(\w+)\s*(\d{4})',
                    r'(?:по|до)\s*(\d{2})\.(\d{2})\.(\d{4})',
                    r'(?:по|до)\s*(\d{2}),\s](\d{2})[,\s',
                    r'(?:по|до)\s*(\d{1,2}),\s](\d{1,2})[,\s',
                    r'(\d{2})\.(\d{2})\.(\d{4})\s*(?:г|год)',
                    r'(\d{2})\.(\d{2})\.(\d{4})',
                    r'(\d{1,2})\.(\d{1,2})\.(\d{4})',
                    ]
            
    if mortgage_date is not None and not pd.isna(mortgage_date):
        for pattern in date_patterns:
            try:
                match = re.search(pattern, text)
                if match:
                    end_date = None
                    
                    if len(match.groups()) == 3 and len(match.group(2)) > 2:
                        month_map = {
                                    'январ': 1, 'феврал': 2, 'март': 3, 'апрел': 4, 'май': 5, 'мая': 5,
                                    'июн': 6, 'июл': 7, 'август': 8, 'сентябр': 9, 'октябр': 10, 
                                    'ноябр': 11, 'декабр': 12
                                    }
                        
                        day = int(match.group(1))
                        month_text = match.group(2).lower()
                        
                    else:
                        day = int(match.group(1))
                        month = int(match.group(2))
                        year = int(match.group(3))

                        if year < 100:
                            year += 2000
                            
                        end_date = datetime(year, month, day)
                    
                    if end_date:
                        start_date = pd.to_datetime(mortgage_date)
                        months_diff = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)
                        
                        if months_diff > 0:
                            return months_diff
                        
            except:
                pass
            
    # Кейс 3: отдельные случаи
    if '360' in text:
        return 360
    elif '300' in text:
        return 300
    elif '240' in text:
        return 240
    elif '180' in text:
        return 180
    elif '120' in text:
        return 120
            
    # Кейс 4: общие положения по ипотеке
    if 'до полного исполнения' in text or 'до полного погашения' in text or 'до полной выплаты' in text:
        return 360
    
    return None


In [ ]:
new_msk['Длительность обременения месяцы'] = new_msk.apply(lambda row: extract_duration_months(row['Условия обременения'], row['Дата обременения']), axis=1) 
new_mo['Длительность обременения месяцы'] = new_mo.apply(lambda row: extract_duration_months(row['Условия обременения'], row['Дата обременения']), axis=1)
new_nm['Длительность обременения месяцы'] = new_nm.apply(lambda row: extract_duration_months(row['Условия обременения'], row['Дата обременения']), axis=1)
new_lo['Длительность обременения месяцы'] = new_lo.apply(lambda row: extract_duration_months(row['Условия обременения'], row['Дата обременения']), axis=1)
new_spb['Длительность обременения месяцы'] = new_spb.apply(lambda row: extract_duration_months(row['Условия обременения'], row['Дата обременения']), axis=1)

#### 1.4.3 Обработка отдельных столбцов

In [ ]:
def data_reform(new_csv: object, region_long: str, region_short: str) -> object:
    
    # ------------------------------------------------------------------------------------------
    # (реновация), (офисы), (регион)
    ''' Меняем наименования ЖК + добавляем к каждому приписку "ЖК " '''

    name_to_change = ' ({})'.format(region_long)

    new_csv = new_csv.copy()
    new_csv['ЖК рус'] = new_csv['ЖК рус'].apply(lambda x: 'ЖК ' + x.replace(' (реновация)', '')
                                                                .replace(' (офисы)', '')
                                                                .replace(name_to_change, '') if isinstance(x, str) else x)


    # ------------------------------------------------------------------------------------------
    ''' 
        Оставляем только квартиры, апартаменты и квартиры + 
        бинаризируем данные (Квартиры == 1, Апартаменты == 0)
    '''
    new_csv = new_csv[(new_csv['Тип помещения'] == 'квартира') | 
                    (new_csv['Тип помещения'] == 'апартамент')]

    new_csv.loc[new_csv['Тип помещения'].astype(str) == 'квартира', 'Тип помещения'] = 1
    new_csv.loc[new_csv['Тип помещения'].astype(str) == 'апартамент', 'Тип помещения'] = 0


    # ------------------------------------------------------------------------------------------
    ''' Районы переименовываем + трансформируем формат данных '''
    name_region = 'Район {}'.format(region_short)
    
    new_csv = new_csv.rename(columns={'Район Город': name_region})

    district_dummies = pd.get_dummies(new_csv[name_region], prefix='Район')
    district_rename = {}

    for district in district_dummies.columns:
        district_rename[district] = district.replace('_', ' ')
    district_dummies = district_dummies.rename(columns=district_rename)

    new_csv = pd.concat([new_csv, district_dummies], axis=1)


    # ------------------------------------------------------------------------------------------
    ''' Регион переименовываем для дамми-переменной "Регион" '''
    
    new_csv.loc[new_csv['Регион'].astype(str) == 'Москва', 'Регион'] = 1
    new_csv.loc[new_csv['Регион'].astype(str) == 'Новая Москва', 'Регион'] = 2
    new_csv.loc[new_csv['Регион'].astype(str) == 'Московская область', 'Регион'] = 3
    new_csv.loc[new_csv['Регион'].astype(str) == 'Санкт-Петербург', 'Регион'] = 4
    new_csv.loc[new_csv['Регион'].astype(str) == 'Ленинградская область', 'Регион'] = 5
    

    # ------------------------------------------------------------------------------------------
    ''' Список Девелоперов нужно превратить в dummy-таблицу '''
    developer_dummies = pd.get_dummies(new_csv['Застройщик ЖК'], prefix='Застройщик')
    developer_rename = {}

    for developer in developer_dummies.columns:
        developer_rename[developer] = developer.replace('Застройщик_', '')
    developer_dummies = developer_dummies.rename(columns=developer_rename)

    new_csv = pd.concat([new_csv, developer_dummies], axis=1)


    # ------------------------------------------------------------------------------------------
    ''' Список Банков (Залогодержателей) нужно превратить в dummy-таблицу '''
    banks_dummies = pd.get_dummies(new_csv['Залогодержатель'], prefix='Банк')
    banks_rename = {}

    for bank in banks_dummies.columns:
        banks_rename[bank] = bank.replace('Банк_', '')
    banks_dummies = banks_dummies.rename(columns=banks_rename)

    new_csv = pd.concat([new_csv, banks_dummies], axis=1)


    # ------------------------------------------------------------------------------------------
    ''' Оставляем только те продажи, площадь которых не превосходит 1200 кв. м '''
    new_csv = new_csv[(new_csv['Площадь'] <= 1200)]


    # ------------------------------------------------------------------------------------------
    ''' Делаем значения этажности < 0 ИЛИ > 85 (самый высокий этаж) пустыми '''
    new_csv.loc[(new_csv['Этаж'] < 0) | (new_csv['Этаж'] > 85), 'Этаж'] = np.nan


    # ------------------------------------------------------------------------------------------
    ''' Оставляем только тех людей, кто купил не более 6 лотов в ЖК '''
    new_csv = new_csv[(new_csv['Купил лотов в ЖК'] <= 6)]


    # ------------------------------------------------------------------------------------------
    ''' 
        Переименовываем наименования значений параметра "Зона" для Региона (в "Зона региона") +  
        пронумировываем от 1 до 4
    '''
    zones_map = {
                '1) внутри СК': 1,
                '2) от СК до ~ТТК': 2,
                '3) от ~ТТК до МКАД': 3,
                '4) Москва за МКАД': 4,
                '5) Н. Москва ближ.': 5,
                '6) Н. Москва дальн.': 6,
                '7) МО ближ.': 7,
                '8) МО ср.': 8,
                '9) МО дальн.': 9,
                '1) СПБ - центр': 10,
                '2) СПБ - КАД': 11,
                '3) СПБ - КАД+': 12,
                '3) СПБ - КАД+': 13,
                '4) СПБ - ЛО': 14
                }
    new_csv['Зона'] = new_csv['Зона'].map(zones_map)


    # ------------------------------------------------------------------------------------------
    ''' Изменяем формат данных для степени готовности ЖК '''
    # Степень готовности
    completion_map = {
                    'котлован': 0.2,
                    'нижние этажи': 0.4,
                    'верхние этажи': 0.6,
                    'идёт отделка': 0.8,
                    'сдан_ГК': 1.0,
                    'Выпущена ПД': np.nan,
                    'Заморожен': np.nan,
                    np.nan: np.nan
                    }
    new_csv['Стадия готовности в дату ДДУ'] = new_csv['Стадия строительства в дату ДДУ'].map(completion_map)

    # Дополнительные флаги
    new_csv['Заморожен'] = new_csv['Стадия строительства в дату ДДУ'] == 'Заморожен'
    new_csv['Выпущена ПД'] = new_csv['Стадия строительства в дату ДДУ'] == 'Выпущена ПД'


    # ------------------------------------------------------------------------------------------
    ''' Тип комнатности трансформируем в числовые показатели '''
    # Словарь соответствия
    rooms_map = { 
                '1': 1,
                '2': 2,
                '3': 3,
                '4': 4,
                'ст': np.nan,
                np.nan: np.nan
                }

    new_csv['Тип Комнатности (обн.)'] = new_csv['Тип Комнатности'].map(rooms_map)
    new_csv['Студия'] = new_csv['Тип Комнатности'] == 'ст'


    # ------------------------------------------------------------------------------------------
    ''' Преобразование отделки '''
    otdelka_map = {
                    'Есть': 1,
                    'Нет': 0,
                    'Неизвестно': np.nan,
                    np.nan: np.nan
                    }
    new_csv['Отделка'] = new_csv['Отделка'].map(otdelka_map)


    # ------------------------------------------------------------------------------------------
    ''' Классы переводим в числовые параметры '''
    class_map = {
                'де-люкс': 5,
                'премиум': 4,
                'бизнес': 3,
                'комфорт': 2,
                'эконом': 1,
                'эконом (панель)': 0.5
                }
    new_csv['класс'] = new_csv['класс'].map(class_map)
    new_csv = new_csv.rename(columns={'класс': 'Класс'})


    # ------------------------------------------------------------------------------------------
    ''' Даты регистрации, обременения и ДДУ трансформируем в три стоблца каждую - день недели, месяц, год '''
    date_columns = ['Дата регистрации', 'Дата обременения', 'Дата ДДУ']

    for col in date_columns:

        # Конвертируем данные в datetime (пробую альтернативный подход)
        new_csv[col] = pd.to_datetime(new_csv[col], errors='coerce')
        
        # Для каждой из дат нужно создать новые столбцы
        if col in new_csv.columns:
            # Выставляется день недели
            new_csv[f"{col} день недели"] = new_csv[col].dt.dayofweek
            # Месяц
            new_csv[f"{col} месяц"] = new_csv[col].dt.month
            # Год
            new_csv[f"{col} год"] = new_csv[col].dt.year
            

    # ------------------------------------------------------------------------------------------
    ''' Дату старта продаж делим в переменные год и месяц '''
    # Преобразуем в строку и разделяем по точке
    new_csv['Старт продаж К (строка)'] = new_csv['Старт продаж К'].astype(str)

    new_csv['Год старта продаж К'] = new_csv['Старт продаж К (строка)'].apply(lambda x: int(x.split('.')[0]) if x != 'nan' else np.nan)
    new_csv['Месяц старта продаж К'] = new_csv['Старт продаж К (строка)'].apply(lambda x: int(x.split('.')[1]) if x != 'nan' else np.nan)

    # Удаляем временный столбец
    new_csv = new_csv.drop(columns=['Старт продаж К', 'Старт продаж К (строка)'], axis=1)


    # ------------------------------------------------------------------------------------------
    ''' Тип обременения '''
    mortgage_map = {
                'ипотека': 1,
                'Ипотека': 1,
                'залог': 2,
                'военная ипотека': 3,
                np.nan: np.nan
                }
    new_csv['Тип обременения'] = new_csv['Тип обременения'].map(mortgage_map)


    # ------------------------------------------------------------------------------------------
    ''' Срок сдачи также разделяем на две переменные: квртал и год '''
    new_csv['Год срока сдачи'] = new_csv['Срок сдачи'].apply(lambda x: int(str(x).split(' кв ')[1]) if str(x) != 'nan' else np.nan)
    new_csv['Квартал срока сдачи'] = new_csv['Срок сдачи'].apply(lambda x: int(str(x).split(' кв ')[0]) if str(x) != 'nan' else np.nan)


    # ------------------------------------------------------------------------------------------
    ''' Обработка сроков обременения (подстановка пустых значений,) '''
    new_csv['Длительность обременения'] = new_csv['Длительность обременения'].fillna(new_csv['Длительность обременения месяцы'])
     
     
    # ------------------------------------------------------------------------------------------
    ''' Обработка покупки физ.лиц / юр.лиц '''
    new_csv['Покупатель ЮЛ'] = np.where(new_csv['Покупатель ЮЛ'].notna() & (new_csv['Покупатель ЮЛ'].str.strip() != ''), 1, 0)
     

    # ------------------------------------------------------------------------------------------
    ''' Избавляемся от старых столбцов '''
    new_csv = new_csv.drop(columns=['Застройщик ЖК',
                                    'Залогодержатель', 
                                    'Район {}'.format(region_short),
                                    'Стадия строительства в дату ДДУ',
                                    'Тип Комнатности',
                                    'Дата регистрации',
                                    'Дата обременения',
                                    'Дата ДДУ',
                                    'Условия обременения',
                                    'Срок сдачи',
                                    'Длительность обременения месяцы'])
    
    
    ''' ПРОЧИЕ ДОРАБОТКИ '''
    # ------------------------------------------------------------------------------------------
    ''' Обработка цены '''
    new_csv['Цена со скидкой'] = new_csv['Цена со скидкой'].apply(lambda x: float(x) if str(x).isnumeric() else np.nan)
    new_csv['Площадь'] = new_csv['Площадь'].apply(lambda x: float(x) if isinstance(x, float) else np.nan)
    
    new_csv['Цена кв.м'] = new_csv['Цена со скидкой'] / new_csv['Площадь']
    
    ''' Столбцы с бинарными параметрами необходимо преобразовать в 1 и 0 '''
    bool_columns = new_csv.select_dtypes(include='bool').columns
    new_csv[bool_columns] = new_csv[bool_columns].astype(int)
    new_csv.replace({True: 1, False: 0}, inplace=True)
    
    new_csv = new_csv.drop(columns=['ID ЖК', 'Цена со скидкой'])
    
    
    return new_csv


#### 1.4.4 Дополнительные корректировки

Они включают в себя:
- Формирование переменной по типу "стоимости квадратного метра"; 
- Изменение формата столбцов (приведение к единому виду для более удобного форматирования);
- Повторное удаление ненужных столбцов и прочие небольшие изменения для более удобного хранения данных

In [ ]:
def final_reform(chunk: object):
    
    # Заполнение пустых столбцов застройщиков и районов 0
    start_col = 'Район Академический'
    end_col = 'ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ КАЗЕННОЕ УЧРЕЖДЕНИЕ "ФЕДЕРАЛЬНОЕ УПРАВЛЕНИЕ НАКОПИТЕЛЬНО-ИПОТЕЧНОЙ СИСТЕМЫ ЖИЛИЩНОГО ОБЕСПЕЧЕНИЯ ВОЕННОСЛУЖАЩИХ"'
    columns_to_fill = chunk.loc[:, start_col:end_col].columns
    
    chunk[columns_to_fill] = chunk[columns_to_fill].fillna(0)
    
    return chunk

#### 1.4.5 Запуск обработок

In [ ]:
# ----------------------------------------------------------------------
# Обработка таблиц
subfinal_df_msk = data_reform(new_msk, 'Москва', 'МСК')
subfinal_df_mo = data_reform(new_mo, 'Московская область', 'МО')
subfinal_df_nm = data_reform(new_nm, 'Новая Москва', 'НМ')
subfinal_df_lo = data_reform(new_lo, 'Ленинградская область', 'ЛО')
subfinal_df_spb = data_reform(new_spb, 'Санкт-Петербург', 'СПБ')

# ----------------------------------------------------------------------
# Очередная чистка памяти
new_msk = None
new_nm = None
new_mo = None
new_spb = None
new_lo = None

In [ ]:
total_subfinal = pd.concat([subfinal_df_msk,
                            subfinal_df_nm,
                            subfinal_df_mo,
                            subfinal_df_spb,
                            subfinal_df_lo], axis=0)
total_subfinal = final_reform(total_subfinal)
total_subfinal.head()

In [ ]:
total_subfinal.to_csv(dcsv_directory + 'reestr_modified_TOTAL.csv', index=False)

  ## 2. ОБЪЕДИНЕНИЕ С ЕРЗ

Происходит поэтапно (по чанкам, так как данные очень большие)

In [ ]:
erz_data = pd.read_csv(data_clean_directory + 'ERZ_X_values.csv')
erz_data = erz_data.rename(columns={'ЖК':'ЖК рус'})
erz_data.head()

In [ ]:
# Загрузка данных по частям
chunk_size = 100000
chunks = pd.read_csv(dcsv_directory + 'reestr_modified_TOTAL.csv', chunksize=chunk_size)
cnt = 0

# Обработка каждого чанка
processed_chunks = []
for chunk in chunks:
    cnt += 1
    print('Чанк №{}'.format(cnt))
    
    new_chunk = pd.merge(chunk, erz_data, on='ЖК рус', how='outer')
    
    special_list = list(chunk.columns)
    special_list.remove('ЖК рус')
    
    new_chunk = new_chunk.dropna(subset=special_list, how='all')
    print(new_chunk)
    
    # Сохранение обработанного чанка
    processed_chunks.append(chunk)
    

# Объединение всех чанков в один DataFrame
df = pd.concat(processed_chunks)

In [ ]:
df.to_csv(dcsv_directory + 'reestr_modified_&_ERZ_X_values_TOTAL.csv', index=False)